In [206]:
#imports
import pandas as pd

In [214]:
# load and present dataset 
df = pd.read_csv('./output.csv')

df.head()

,u_cost_rel,u_consent_rel,u_content_rel,s_cost_rel,s_consent_rel,s_content_rel,score_binary_2c,score_binary_3c,score_562_2c,score_562_3c,score_532_2c,score_532_3c,score_524_2c,score_524_3c,score_564_2c,score_564_3c,Unnamed: 16
0,0.0,0.1,0.9,0.0,0.1,0.9,9000,9000,9025,9025,9025,9025,9000,9000,9025,9025,NaN
1,0.0,0.1,0.9,0.0,0.2,0.8,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,NaN
2,0.0,0.1,0.9,0.0,0.3,0.7,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,NaN
3,0.0,0.1,0.9,0.0,0.4,0.6,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,NaN
4,0.0,0.1,0.9,0.0,0.5,0.5,9000,9000,9000,9000,9000,9000,9000,9000,9000,9000,NaN


In [215]:
# Create additional columns
df['diff_562_2c'] = df['score_562_2c'] - df['score_binary_2c']
df['diff_562_3c'] = df['score_562_3c'] - df['score_binary_3c']

df['diff_532_2c'] = df['score_532_2c'] - df['score_binary_2c']
df['diff_532_3c'] = df['score_532_3c'] - df['score_binary_3c']

df['diff_524_2c'] = df['score_524_2c'] - df['score_binary_2c']
df['diff_524_3c'] = df['score_524_3c'] - df['score_binary_3c']

df['diff_564_2c'] = df['score_564_2c'] - df['score_binary_2c']
df['diff_564_3c'] = df['score_564_3c'] - df['score_binary_3c']

# function to calculate the absolute difference range
def get_diff_abs(diff):
    if diff == 0:
        return "0"
    elif 0 < diff <= 100:
        return "1to100"
    elif 100 < diff <= 500:
        return "101to500"
    elif 500 < diff <= 1000:
        return "501to1000"
    elif 1000 < diff <= 2000:
        return "1001to2000"
    elif diff > 2000:
        return "greaterThan2000"
    else:
        return "invalid"

# Calculate absolute difference ranges
df['abs_562_2c'] = df['diff_562_2c'].apply(get_diff_abs)
df['abs_562_3c'] = df['diff_562_3c'].apply(get_diff_abs)

df['abs_532_2c'] = df['diff_532_2c'].apply(get_diff_abs)
df['abs_532_3c'] = df['diff_532_3c'].apply(get_diff_abs)

df['abs_524_2c'] = df['diff_524_2c'].apply(get_diff_abs)
df['abs_524_3c'] = df['diff_524_3c'].apply(get_diff_abs)

df['abs_564_2c'] = df['diff_564_2c'].apply(get_diff_abs)
df['abs_564_3c'] = df['diff_564_3c'].apply(get_diff_abs)

# function to calculate the relative difference 
def get_diff_rel(diff):
    return 
# Calculate the relative difference
#df['diff_2c_rel'] = round(df['diff_2c']/df['score_binary_2c'],3)
#df['diff_3c_rel'] = round(df['diff_3c']/df['score_binary_3c'],3)
df['rel_562_2c'] = round(df['diff_562_2c']/df['score_binary_2c'],3)
df['rel_562_3c'] = round(df['diff_562_3c']/df['score_binary_3c'],3)

df['rel_532_2c'] = round(df['diff_532_2c']/df['score_binary_2c'],3)
df['rel_532_3c'] = round(df['diff_532_3c']/df['score_binary_3c'],3)

df['rel_524_2c'] = round(df['diff_524_2c']/df['score_binary_2c'],3)
df['rel_524_3c'] = round(df['diff_524_3c']/df['score_binary_3c'],3)

df['rel_564_2c'] = round(df['diff_564_2c']/df['score_binary_2c'],3)
df['rel_564_3c'] = round(df['diff_564_3c']/df['score_binary_3c'],3)


# Display the DataFrame (which will be rendered as a table)
df.head()

,u_cost_rel,u_consent_rel,u_content_rel,s_cost_rel,s_consent_rel,s_content_rel,score_binary_2c,score_binary_3c,score_562_2c,score_562_3c,...,abs_564_2c,abs_564_3c,rel_562_2c,rel_562_3c,rel_532_2c,rel_532_3c,rel_524_2c,rel_524_3c,rel_564_2c,rel_564_3c
0,0.0,0.1,0.9,0.0,0.1,0.9,9000,9000,9025,9025,...,1to100,1to100,0.003,0.003,0.003,0.003,0.0,0.0,0.003,0.003
1,0.0,0.1,0.9,0.0,0.2,0.8,9000,9000,9000,9000,...,0,0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000
2,0.0,0.1,0.9,0.0,0.3,0.7,9000,9000,9000,9000,...,0,0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000
3,0.0,0.1,0.9,0.0,0.4,0.6,9000,9000,9000,9000,...,0,0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000
4,0.0,0.1,0.9,0.0,0.5,0.5,9000,9000,9000,9000,...,0,0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000


In [209]:
df.columns



Index(['u_cost_rel', 'u_consent_rel', 'u_content_rel', 's_cost_rel',
       's_consent_rel', 's_content_rel', 'score_binary_2c', 'score_binary_3c',
       'score_562_2c', 'score_562_3c', 'score_532_2c', 'score_532_3c',
       'score_524_2c', 'score_524_3c', 'score_564_2c', 'score_564_3c',
       'Unnamed: 16', 'diff_562_2c', 'diff_562_3c', 'diff_532_2c',
       'diff_532_3c', 'diff_524_2c', 'diff_524_3c', 'diff_564_2c',
       'diff_564_3c', 'abs_562_2c', 'abs_562_3c', 'abs_532_2c', 'abs_532_3c',
       'abs_524_2c', 'abs_524_3c', 'abs_564_2c', 'abs_564_3c', 'rel_562_2c',
       'rel_562_3c', 'rel_532_2c', 'rel_532_3c', 'rel_524_2c', 'rel_524_3c',
       'rel_564_2c', 'rel_564_3c'],
      dtype='object')

In [210]:
# distribution of absolute differences
print('How many points are gained when negotiating consent and content with more detail?')

rel_gains_df = pd.DataFrame()

rel_gains_df['abs_distribution'] = ['0', '1to100', '101to500', '501to1000', '1001to2000', "greaterThan2000"]

# Iterate over each column name
for column_name in ['abs_562_2c', 'abs_562_3c', 'abs_532_2c', 'abs_532_3c', 'abs_524_2c', 'abs_524_3c', 'abs_564_2c', 'abs_564_3c',  ]:
    values = df[column_name].value_counts().values
    if (values.size == 1):
        values = [values[0], 0, 0 ,0, 0, 0]
    elif(values.size == 5):
        values = values[0].append(0)

    rel_gains_df[column_name] = values

# Display the resulting DataFrame
rel_gains_df

How many points are gained when negotiating consent and content with more detail?


,abs_distribution,abs_562_2c,abs_562_3c,abs_532_2c,abs_532_3c,abs_524_2c,abs_524_3c,abs_564_2c,abs_564_3c
0,0,1171,1182,1171,1189,1681,1681,1171,1182
1,1to100,236,241,259,261,0,0,236,241
2,101to500,200,198,178,171,0,0,200,198
3,501to1000,60,46,59,47,0,0,60,46
4,1001to2000,13,13,13,12,0,0,13,13
5,greaterThan2000,1,1,1,1,0,0,1,1


Observations:
- most negotiations gained zero points 

????????? abs_524_2c	abs_524_3c ARE EMPTY ---> ??!?!?!?!?!?!
????????? abs_564_2c	abs_564_3c ARE SAME AS bs_562_2c	abs_562_3c	

- 236 to 261 have small gains from one to 100 points
- 171 to 200 have gains from 100 to 500 points
- very little (~13) have gains between 1000 and 2000
- an outlier seems to exist in every case

Interpretation:
- the granularization of consent and content lead to zero gained points in most negotiations


In [219]:
# distribution of absolute differences
print('How much is gained when negotiating consent and content with more detail?')

rel_gains_df = pd.DataFrame()

rel_gains_df['rel_distribution'] = ['Mean', 'Standard Deviation', 'Minimum', '25th Percentile', 'Median', "75th Percentile", "90th Percentile", "95th Percentile", "Maximum"]

# Iterate over each column name
for column_name in ['rel_562_2c', 'rel_562_3c', 'rel_532_2c', 'rel_532_3c', 'rel_524_2c', 'rel_524_3c', 'rel_564_2c', 'rel_564_3c',  ]:
    mean = round(df[column_name].mean(), 3)
    std = round(df[column_name].std(), 3)
    min_value = round(df[column_name].min(), 3)
    q25 = round(df[column_name].quantile(0.25), 3)
    median = round(df[column_name].median(), 3)
    q75 = round(df[column_name].quantile(0.75), 3)
    q90 = round(df[column_name].quantile(0.90), 3)
    q95 = round(df[column_name].quantile(0.95), 3)
    max_value = round(df[column_name].max(), 3)

    # add column
    rel_gains_df[column_name] = [mean, std, min, q25, median, q75, q90, q95, max]

# Display the resulting DataFrame
rel_gains_df

How much is gained when negotiating consent and content with more detail?


,rel_distribution,rel_562_2c,rel_562_3c,rel_532_2c,rel_532_3c,rel_524_2c,rel_524_3c,rel_564_2c,rel_564_3c
0,Mean,0.028,0.019,0.028,0.018,0.000,0.000,0.028,0.019
1,Standard Deviation,0.109,0.083,0.109,0.082,0.000,0.000,0.109,0.083
2,Minimum,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,25th Percentile,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,Median,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
5,75th Percentile,0.006,0.003,0.006,0.002,0.000,0.000,0.006,0.003
6,90th Percentile,0.066,0.041,0.064,0.038,0.000,0.000,0.066,0.041
7,95th Percentile,0.143,0.095,0.143,0.088,0.000,0.000,0.143,0.095
8,Maximum,2.025,2.025,2.025,2.025,2.025,2.025,2.025,2.025


Observations:
- First improvements observed in the 75th percentile 
- All seem to have the same maximum outlier, probably leveraging the mean
- rel_524_2c, rel_524_3c show no improvement
- min ~10% improvement in 95th percentile

Interpretation:
- The granuralization of consent and content into more than two resolutions did increase gains a lot

In [227]:
# Relation between prioritization of issues (relevancies) and relative gains

# Which combination creates the outlier of over 200% relative gains?
df[df['rel_562_2c'] > 2][['u_cost_rel', 'u_consent_rel', 'u_content_rel', 's_cost_rel', 's_consent_rel', 's_content_rel']]


,u_cost_rel,u_consent_rel,u_content_rel,s_cost_rel,s_consent_rel,s_content_rel
336,0.0,0.9,0.1,0.0,0.9,0.1


Observations:
- The outlier is based on extreme interest in consent

Interpretation:
- When both user and site only care about privacy/consent and not about content or cost, this happens

In [252]:
# Create columns to analyze combinations of issue relevancies
def map_relevancy_dec_to_category(value):
    if value < 0.3:
        return 'L'
    elif value <= 0.5:
        return 'M'
    else:
        return 'H'

def map_relevancies_to_string(cost, consent, content):
    cost_string = map_relevancy_dec_to_category(cost)
    consent_string = map_relevancy_dec_to_category(consent)
    content_string = map_relevancy_dec_to_category(content)
    return cost_string + consent_string + content_string

def map_user_relevancies_to_string(row):
    return map_relevancies_to_string(row['u_cost_rel'], row['u_consent_rel'], row['u_content_rel'])

def map_sites_relevancies_to_string(row):
    return map_relevancies_to_string(row['s_cost_rel'], row['s_consent_rel'], row['s_content_rel'])


df['u_issue_rel'] = df.apply(map_user_relevancies_to_string, axis=1)
df['s_issue_rel'] = df.apply(map_sites_relevancies_to_string, axis=1) # doesnt yet exist. 

# Print head for verification
df.head()

,u_cost_rel,u_consent_rel,u_content_rel,s_cost_rel,s_consent_rel,s_content_rel,score_binary_2c,score_binary_3c,score_562_2c,score_562_3c,...,rel_562_2c,rel_562_3c,rel_532_2c,rel_532_3c,rel_524_2c,rel_524_3c,rel_564_2c,rel_564_3c,u_issue_rel,s_issue_rel
0,0.0,0.1,0.9,0.0,0.1,0.9,9000,9000,9025,9025,...,0.003,0.003,0.003,0.003,0.0,0.0,0.003,0.003,LLH,LLH
1,0.0,0.1,0.9,0.0,0.2,0.8,9000,9000,9000,9000,...,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,LLH,LLH
2,0.0,0.1,0.9,0.0,0.3,0.7,9000,9000,9000,9000,...,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,LLH,LMH
3,0.0,0.1,0.9,0.0,0.4,0.6,9000,9000,9000,9000,...,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,LLH,LMH
4,0.0,0.1,0.9,0.0,0.5,0.5,9000,9000,9000,9000,...,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,LLH,LMM


In [267]:
# Which combination of user relavancies creates the highest (90th percentile) relative gains?
# EXCLUDE: rel_524_2c, rel_524_3c, because no gains
rel_columns = ['rel_562_2c', 'rel_562_3c', 'rel_532_2c', 'rel_532_3c', 'rel_564_2c', 'rel_564_3c']

# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame()

# Iterate over each column
for column_name in rel_columns:
    # Calculate the 90th percentile value
    top_ten_percentile_value = df[column_name].quantile(0.9)
    
    # Filter the DataFrame based on the condition and select 'u_issue_rel' column
    top_percentile_df = df[df[column_name] > top_ten_percentile_value][['u_issue_rel']]
    
    # Calculate value counts for 'u_issue_rel' and reset the index
    counts = top_percentile_df['u_issue_rel'].value_counts()
    
    # Append counts for current column to result DataFrame
    result_df[column_name] = counts

result_df

,rel_562_2c,rel_562_3c,rel_532_2c,rel_532_3c,rel_564_2c,rel_564_3c
LHL,71,44,70,44,71,44
LHM,46,32,44,32,46,32
LMM,25,36,25,38,25,36
MML,18,41,18,38,18,41
LMH,2,6,2,6,2,6
MMM,2,9,2,8,2,9


Observations:
- LHL, LHM, LMM and MML are the most common combinations of user relevancies
- LMH and MMM are already far less common
- in the most and secondmost common (LHL, LHM) consent preferences are H (high) or at least 0.5
- in the most common combinations, consent preferences are always >L or M/H

Interpretation:
- The top four combinations occur for extremly privacy concerned users, that care more about their consent than about cost and content combined

In [268]:
# Which combination of sites relavancies creates the highest (90th percentile) relative gains?
# EXCLUDE: rel_524_2c, rel_524_3c, because no gains
rel_columns = ['rel_562_2c', 'rel_562_3c', 'rel_532_2c', 'rel_532_3c', 'rel_564_2c', 'rel_564_3c']

# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame()

# Iterate over each column
for column_name in rel_columns:
    # Calculate the 90th percentile value
    top_ten_percentile_value = df[column_name].quantile(0.9)
    
    # Filter the DataFrame based on the condition and select 'u_issue_rel' column
    top_percentile_df = df[df[column_name] > top_ten_percentile_value][['s_issue_rel']]
    
    # Calculate value counts for 'u_issue_rel' and reset the index
    counts = top_percentile_df['s_issue_rel'].value_counts()
    
    # Append counts for current column to result DataFrame
    result_df[column_name] = counts

result_df

,rel_562_2c,rel_562_3c,rel_532_2c,rel_532_3c,rel_564_2c,rel_564_3c
MML,39,24.0,39,26.0,39,24.0
LHL,37,50.0,34,46.0,37,50.0
LMM,28,45.0,28,44.0,28,45.0
LHM,23,37.0,23,37.0,23,37.0
MMM,15,8.0,15,8.0,15,8.0
HLL,10,NaN,10,NaN,10,NaN
LMH,6,4.0,6,5.0,6,4.0
MLM,6,NaN,6,NaN,6,NaN


Observations:
- as for the user LHL, LHM, LMM and MML are the most common combinations of user relevancies
- the other combinations are less common
- in the most common combinations, consent preferences are always >L or M/H

Interpretation:
- The top four combinations occur when a site is way more interested in consent, that granting content or getting to some cost